In [10]:
#!/usr/bin/env python

# import all dependencies
import joblib
import flask
import pandas as pd
import os
from typing import Dict, Union, Tuple

from flask_ngrok import run_with_ngrok

In [11]:
# setup
BASE_PATH = '..'
FEATURES = sorted([
    "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
    "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
    "pH", "sulphates", "alcohol",
])

# load model
model = joblib.load(os.path.join(BASE_PATH, 'models', 'regression_model'), 'r')

# initiate flask app
app = flask.Flask(__name__)
run_with_ngrok(app)

In [12]:
# request validation function

def validate_request(json_data: Dict[str, Dict[str, Union[list, float, int]]]) -> Tuple[bool, str]:
    """
    Check request data for validity.
    :param json_data: input json from request body
    :return: Tuple[boolean, string], (True, 'ok') if data is valid, (False, <error message>) otherwise
    """
    # check data structure
    if not isinstance(json_data, dict):
        return False, f"data isn't a dictionary, it's type is {type(json_data)}"
    if 'features' not in json_data.keys():
        return False, "No 'features' key"
    if not isinstance(json_data['features'], dict):
        return False, "data['features'] isn't a dictionary"

    # check that features have the same count
    feature_num = None
    for feature in FEATURES:
        if feature not in json_data['features'].keys():
            return False, f"There is no feature {feature}"
        if isinstance(json_data['features'][feature], (int, float)):
            if feature_num is None:
                feature_num = 1
            else:
                if feature_num != 1:
                    return False, "Mismatch in feature count"
        elif isinstance(json_data['features'][feature], list):
            if feature_num is None:
                feature_num = len(json_data['features'][feature])
            else:
                if feature_num != len(json_data['features'][feature]):
                    return False, "Mismatch in feature count"
        else:
            return False, "Features must bu of type 'int, float' - for single prediction, 'list' for multiple"

    return True, "Ok"

In [13]:
def prepare_data(json_data: Dict[str, Dict[str, Union[list, int, float]]]) -> pd.DataFrame:
    """
    Helper function to create appropriate data format for ml model 
    
    :param json_data: request data
    :return: pd.DataFrame with all needed features in right order
    """
    return pd.DataFrame(
        data={
            feature: [json_data['features'][feature]]
            for feature in FEATURES
        }
    )

In [14]:
@app.route('/', methods=['POST'])
def make_prediction():
    # get data from request
    json_data = flask.request.get_json()
    # validate data
    is_data_valid, validator_message = validate_request(json_data)
    if not is_data_valid:
        return flask.jsonify(
            error_message=(
                f"Wrong request structure.\n\r"
                f"You should have dict('features': dict(<all features>: feature value))\n\r"
                f"Required feature list: {FEATURES}\n\r",
                f"Validation message: {validator_message}"
            ),
        )
    # create df
    df = prepare_data(json_data)
    # run model prediction
    predictions = model.predict(df)
    # return answer
    return flask.jsonify(
        predictions=list(map(float, predictions)),
    )

In [15]:
# run flack application
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 1254, in run
    self.function(*self.args, **self.kwargs)
  File "/data/Education/SIT/DataSceince/pyenv/lib/python3.8/site-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/data/Education/SIT/DataSceince/pyenv/lib/python3.8/site-packages/flask_ngrok.py", line 31, in _run_ngrok
    ngrok = subprocess.Popen([executable, 'http', '5000'])
  File "/usr/lib/python3.8/subprocess.py", line 854, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/lib/python3.8/subprocess.py", line 1702, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
PermissionError: [Errno 13] Permission denied: '/tmp/ngrok/ngrok'
127.0.0.1 - - [22/Apr/2021 09